# 01 - BaT Scraping

Scrape completed Porsche 911 auctions from Bring a Trailer.

**Outputs:**
- `data/raw/bat_listings.parquet` - Raw scraped data

In [1]:
# delete bad data if needed
!rm data/raw/bat_listings.parquet

rm: data/raw/bat_listings.parquet: No such file or directory


In [2]:
import logging
import pandas as pd
from pathlib import Path

from price_analysis.scraping import fetch_auctions, validate_scraped_data, DataQualityError
from price_analysis.scraping.bat import listings_to_dataframe

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
# Paths
DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "raw"
RAW_DIR.mkdir(parents=True, exist_ok=True)

OUTPUT_PATH = RAW_DIR / "bat_listings.parquet"

## Configure Scraping

Adjust search queries and pagination as needed.

In [4]:
# Search queries - multiple generation-specific searches to get more results
# BaT seems to limit how many results a single search returns
QUERIES = [
    "Porsche 996",  # 1999-2004 (996.1 and 996.2)
    # "Porsche 997",  # 2005-2012 (997.1 and 997.2)
    # "Porsche 991",  # 2012-2019 (991.1 and 991.2)
    # "Porsche 992",  # 2020+ (992.1 and 992.2)
    # "Porsche 911",  # Catch air-cooled + anything else missed
]

# Scraping parameters
MAX_CLICKS = 100  # Number of "Show More" button clicks (each loads ~24 listings)
DELAY = 2.5  # Seconds between requests (be polite!)
HEADLESS = True  # Set False to see browser for debugging

## Debug: Inspect BaT Page Structure (Optional)

Skip this section - fixture files already saved to `tests/fixtures/`.
Only re-run if BaT changes their DOM structure.

In [5]:
# Debug: Save sample pages for selector inspection
import time
from price_analysis.scraping import create_driver, save_debug_page

# Create driver (set HEADLESS=False above to watch)
driver = create_driver(headless=HEADLESS)

try:
    # 1. Fetch search results page
    search_url = "https://bringatrailer.com/auctions/results/?s=Porsche+911"
    logger.info(f"Fetching search page: {search_url}")
    driver.get(search_url)
    time.sleep(3)  # Wait for JS to load
    save_debug_page(driver, "bat_search_page", output_dir="../tests/fixtures")

    # 2. Fetch a specific listing page (992.1)
    listing_url = "https://bringatrailer.com/listing/2020-porsche-911-carrera-4s-coupe-26/"
    logger.info(f"Fetching listing: {listing_url}")
    driver.get(listing_url)
    time.sleep(3)
    save_debug_page(driver, "bat_listing_992", output_dir="../tests/fixtures")

    # 3. Fetch another listing (997.2) for variety
    listing_url_997 = "https://bringatrailer.com/listing/2009-porsche-911-carrera-4s-coupe-28/"
    logger.info(f"Fetching listing: {listing_url_997}")
    driver.get(listing_url_997)
    time.sleep(3)
    save_debug_page(driver, "bat_listing_997", output_dir="../tests/fixtures")

    logger.info("Debug pages saved to tests/fixtures/")

finally:
    driver.quit()

INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [/Users/flatljan/.wdm/drivers/chromedriver/mac64/143.0.7499.169/chromedriver-mac-arm64/chromedriver] found in cache
INFO:__main__:Fetching search page: https://bringatrailer.com/auctions/results/?s=Porsche+911
INFO:price_analysis.scraping.bat:Saved debug page to ../tests/fixtures/bat_search_page.html
INFO:__main__:Fetching listing: https://bringatrailer.com/listing/2020-porsche-911-carrera-4s-coupe-26/
INFO:price_analysis.scraping.bat:Saved debug page to ../tests/fixtures/bat_listing_992.html
INFO:__main__:Fetching listing: https://bringatrailer.com/listing/2009-porsche-911-carrera-4s-coupe-28/
INFO:price_analysis.scraping.bat:Saved debug page to ../tests/fixtures/bat_listing_997.html
INFO:__main__:Debug pages saved to tests/fixtures/


## Run Scraper

This will take a while depending on MAX_PAGES. Each page + listing takes ~3-5 seconds.

In [6]:
all_listings = []

for query in QUERIES:
    logger.info(f"Scraping: {query}")
    listings = fetch_auctions(
        query=query,
        max_clicks=MAX_CLICKS,
        delay=DELAY,
        headless=HEADLESS,
    )
    all_listings.extend(listings)
    logger.info(f"Found {len(listings)} listings for '{query}'")

logger.info(f"Total listings scraped: {len(all_listings)}")

INFO:__main__:Scraping: Porsche 996
INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [/Users/flatljan/.wdm/drivers/chromedriver/mac64/143.0.7499.169/chromedriver-mac-arm64/chromedriver] found in cache
INFO:price_analysis.scraping.bat:Fetching search results: https://bringatrailer.com/auctions/results/?s=Porsche+996
INFO:price_analysis.scraping.bat:No more 'Show More' button available - reached end of results
INFO:price_analysis.scraping.bat:Found completed auctions container - searching within it
INFO:price_analysis.scraping.bat:Found 24 listing cards on page
INFO:price_analysis.scraping.bat:Filtered out 14 non-car listings (parts/wheels/etc)
INFO:price_analysis.scraping.bat:Found 10 unique listings after 0 'Show More' clicks
INFO:price_analysis.scraping.bat:Found 10 unique listings to fetch
INFO:price_analysis.scraping.bat:Processing listing 1/10
INFO:price_ana

## Convert to DataFrame and Save

In [7]:
df = listings_to_dataframe(all_listings)

# Run data quality checks and filter low-price listings (<$10k)
# Returns filtered DataFrame, raises DataQualityError if checks fail
df = validate_scraped_data(df)

display(df.head(10))
print(f"\nShape: {df.shape}")

INFO:price_analysis.scraping.bat:Filtered 2 listings below $10,000 (likely parts/salvage)
INFO:price_analysis.scraping.bat:Data quality checks passed: 8 listings, 8 with prices


,listing_url,title_raw,sale_price,sale_date,model_year,generation,trim,transmission,mileage,color,location
1,https://bringatrailer.com/listing/1999-porsche...,22k-Mile 1999 Porsche 911 Carrera 4 Coupe 6-Speed,45000,2025-12-18,1999.0,996.1,Carrera 4,Manual,22000.0,Arena Red Metallic,"Newtown, Pennsylvania 18940"
2,https://bringatrailer.com/listing/2002-porsche...,15k-Mile 2002 Porsche 911 GT2,231000,2025-12-17,2002.0,996.2,None,Manual,15000.0,None,"Burlingame, California 94010"
3,https://bringatrailer.com/listing/2004-porsche...,2004 Porsche 911 Turbo Cabriolet 6-Speed,48500,2025-12-19,2004.0,996.2,Turbo,Manual,65000.0,Arctic Silver Metallic,"Portland, Oregon 97227"
4,https://bringatrailer.com/listing/1999-porsche...,1999 Porsche 911 Carrera Coupe 6-Speed,28650,2025-12-24,1999.0,996.1,Carrera,Manual,71000.0,Vesuvio Metallic,"San Luis Obispo, California 93401"
6,https://bringatrailer.com/listing/2000-porsche...,40k-Mile 2000 Porsche 911 Carrera Cabriolet 6-...,20996,2025-12-10,2000.0,996.1,Carrera,Manual,40000.0,Ocean Blue Metallic,"West Palm Beach, Florida 33407"
7,https://bringatrailer.com/listing/2002-porsche...,2002 Porsche 911 Carrera Cabriolet 6-Speed,21000,2025-12-22,2002.0,996.2,Carrera,Manual,99000.0,Lapis Blue Metallic,"Boring, Oregon 97009"
8,https://bringatrailer.com/listing/2003-porsche...,2003 Porsche 911 Carrera Targa 6-Speed,28000,2025-12-19,2003.0,996.2,Targa,Manual,89000.0,Midnight Blue Metallic,"Wayne, New Jersey 07470"
9,https://bringatrailer.com/listing/2001-porsche...,18k-Mile 2001 Porsche 911 Carrera 4 Cabriolet ...,38000,2025-12-12,2001.0,996.1,Carrera 4,Manual,18000.0,Black,"Pompano Beach, Florida 33064"



Shape: (8, 11)


In [8]:
# Append to existing if present, otherwise create new
if OUTPUT_PATH.exists():
    existing = pd.read_parquet(OUTPUT_PATH)
    df = pd.concat([existing, df], ignore_index=True)
    df = df.drop_duplicates(subset=["listing_url"], keep="last")
    logger.info(f"Merged with existing data: {len(df)} total listings")

df.to_parquet(OUTPUT_PATH, index=False)
logger.info(f"Saved to {OUTPUT_PATH}")

INFO:__main__:Merged with existing data: 32 total listings
INFO:__main__:Saved to ../data/raw/bat_listings.parquet


## Quick Inspection

In [9]:
print("Counts by generation:")
display(df["generation"].value_counts(dropna=False))

Counts by generation:


generation
None     12
996.2     6
996.1     5
997.1     3
991.2     2
992.1     2
992.2     1
997.2     1
Name: count, dtype: int64

In [10]:
print("Counts by trim:")
display(df["trim"].value_counts(dropna=False))

Counts by trim:


trim
Carrera        9
None           5
Turbo          5
Carrera 4      4
Targa          2
Carrera 4S     2
Carrera S      1
GT3            1
GT3 Touring    1
GT2 RS         1
Turbo S        1
Name: count, dtype: int64

In [11]:
print("Counts by transmission:")
display(df["transmission"].value_counts(dropna=False))

Counts by transmission:


transmission
Manual       27
None          2
PDK           2
Automatic     1
Name: count, dtype: int64

In [12]:
# Check parsing quality - how many have all required fields?
required = ["sale_price", "model_year", "generation", "trim", "transmission", "mileage"]
complete = df[required].notna().all(axis=1).sum()
print(
    f"\nListings with all required fields: {complete} / {len(df)} ({complete / len(df) * 100:.1f}%)"
)


Listings with all required fields: 18 / 32 (56.2%)


In [13]:
# Sample some listings to verify parsing
print("Sample listings for manual verification:")
sample = df.sample(min(5, len(df)))
for _, row in sample.iterrows():
    print(f"\n{row['title_raw']}")
    print(
        f"  Parsed: {row['model_year']} {row['generation']} {row['trim']} ({row['transmission']})"
    )
    print(f"  Price: ${row['sale_price']:,}" if pd.notna(row["sale_price"]) else "  Price: N/A")
    print(f"  Mileage: {row['mileage']:,}" if pd.notna(row["mileage"]) else "  Mileage: N/A")

Sample listings for manual verification:

2024 Porsche 911 GT3 Touring 6-Speed
  Parsed: 2024.0 992.2 GT3 Touring (Manual)
  Price: $272,000
  Mileage: 3,000.0

Twin-Turbocharged 3.0L-Powered 1974 Porsche 911 Cabriolet Conversion G50
  Parsed: nan None None (Manual)
  Price: $77,500
  Mileage: 47,000.0

100-Mile Meissen Blue 2019 Porsche 911 Speedster
  Parsed: 2019.0 991.2 None (Manual)
  Price: $531,000
  Mileage: 100.0

15k-Mile 2009 Porsche 911 Carrera 4S Coupe 6-Speed
  Parsed: 2009.0 997.2 Carrera 4S (Manual)
  Price: $105,000
  Mileage: 15,000.0

1999 Porsche 911 Carrera Coupe 6-Speed
  Parsed: 1999.0 996.1 Carrera (Manual)
  Price: $28,650
  Mileage: 71,000.0
